In [1]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from spring_example import get_spring_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf
from scipy.integrate import solve_ivp
import pickle

# Generate data

In [6]:
training_data = get_spring_data(100)
validation_data = get_spring_data(10)

In [7]:
len(training_data['t'])

289

# Set up model and training parameters

In [8]:
params = {}

params['input_dim'] = training_data['x'].shape[-1]
params['latent_dim'] = 2
params['poly_order'] = 2
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] ='glorot_normal'
# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = 5e-3
params['loss_weight_sindy_z'] = 5e-3
params['loss_weight_sindy_regularization'] = 1e-5


params['activation'] = 'sigmoid'
params['widths'] = [128,64]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 289
params['learning_rate'] = 1e-4

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [ ]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'experiment_results_'+str(i) + '.pkl'

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    
    # Assuming results_dict and params are dictionaries
    data_to_append = {**results_dict, **params}

    # Create a Series from the data_to_append dictionary
    series_to_append = pd.Series(data_to_append)

    # Save the Series as a pickle file
    with open('experiment_results_4_' + str(i) + '.pkl', 'wb') as file:
        pickle.dump(series_to_append, file)
    

EXPERIMENT 0
TRAINING
Epoch 0
   training loss 0.03280152752995491, (0.03213607, 0.13103251, 0.0008843425, 0.5872416)
   validation loss 0.09752825647592545, (0.09620917, 0.13268586, 0.129958, 0.5872416)
decoder loss ratio: 1.498605, decoder SINDy loss  ratio: 1.000488
Epoch 100
   training loss 4.089771391591057e-05, (3.5415116e-05, 8.355535e-05, 9.3826515e-05, 0.45956865)
   validation loss 0.002460563788190484, (0.0022010808, 0.02802693, 0.022950554, 0.45956865)
decoder loss ratio: 0.034285, decoder SINDy loss  ratio: 0.176686
Epoch 200
   training loss 3.843058584607206e-05, (3.3168948e-05, 8.260125e-05, 0.000114300834, 0.4277127)
   validation loss 0.0018977689323946834, (0.001772716, 0.00893832, 0.015216852, 0.4277127)
decoder loss ratio: 0.027613, decoder SINDy loss  ratio: 0.117148
Epoch 300
   training loss 3.286408900748938e-05, (2.7344488e-05, 0.000110817076, 0.00016471106, 0.41419592)
   validation loss 0.0015613127034157515, (0.0014502131, 0.0078026964, 0.013588837, 0.4141